In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set parameters
input_shape = (150, 150, 3)  # Input shape of images (150x150 pixels with 3 color channels)
batch_size = 32
epochs = 25  # You mentioned using 25 epochs

# Create a sequential model
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output to feed into the dense layers
model.add(Flatten())

# Fully connected dense layer
model.add(Dense(512, activation='relu'))

# Dropout for regularization to prevent overfitting
model.add(Dropout(0.5))

# Output layer (1 node with sigmoid activation for binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()



c:\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:

# Use ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,               # Rescale pixel values from 0-255 to 0-1
    rotation_range=40,            # Randomly rotate images
    width_shift_range=0.2,        # Randomly translate images horizontally
    height_shift_range=0.2,       # Randomly translate images vertically
    shear_range=0.2,              # Randomly apply shearing transformations
    zoom_range=0.2,               # Randomly zoom in on images
    horizontal_flip=True,         # Randomly flip images horizontally
    fill_mode='nearest'           # Fill in missing pixels with the nearest value
)

# Validation data should only be rescaled
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training data
train_generator = train_datagen.flow_from_directory(
    'CNN_dataset/training_set',          # Path to training data directory
    target_size=(150, 150),        # Resize images to 150x150
    batch_size=batch_size,
    class_mode='binary'            # Binary classification (cat vs. dog)
)

# Load and preprocess the validation data
validation_generator = validation_datagen.flow_from_directory(
    'CNN_dataset/test_set',     # Path to validation data directory
    target_size=(150, 150),        # Resize images to 150x150
    batch_size=batch_size,
    class_mode='binary'            # Binary classification (cat vs. dog)
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the model after training
model.save('CNN_model/cnn_cat_or_dog_v3.0.h5')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25


c:\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 67s 261ms/step - accuracy: 0.4985 - loss: 0.6975 - val_accuracy: 0.5071 - val_loss: 0.6859
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 299us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4375 - val_loss: 0.6718
Epoch 3/25


c:\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


250/250 ━━━━━━━━━━━━━━━━━━━━ 64s 255ms/step - accuracy: 0.5548 - loss: 0.6855 - val_accuracy: 0.5121 - val_loss: 0.6923
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 149us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6250 - val_loss: 0.6713
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 63s 247ms/step - accuracy: 0.5511 - loss: 0.6871 - val_accuracy: 0.5489 - val_loss: 0.6821
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 125us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5625 - val_loss: 0.6546
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 63s 250ms/step - accuracy: 0.5739 - loss: 0.6755 - val_accuracy: 0.6361 - val_loss: 0.6471
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 169us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.5667
Epoch 9/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 63s 250ms/step - accuracy: 0.6208 - loss: 0.6551 - val_accuracy: 0.5590 - val_loss: 0.6786
Epoch 10/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 140us/step - accuracy: 0.0000

### Load Model and Predict

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('CNN_model/cnn_cat_or_dog_v3.0.h5')

# Define the image size (should match the input size used during training)
img_width, img_height = 150, 150

# Load and preprocess the image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create a batch dimension
    img_array = img_array / 255.0  # Scale pixel values to [0, 1]
    return img_array

# Path to the image you want to test
img_path = 'CNN_dataset/single_prediction/cat_or_dog_4.jpg'

# Preprocess the image
preprocessed_image = load_and_preprocess_image(img_path)

# Make predictions
predictions = model.predict(preprocessed_image)

# Get the prediction result
if predictions[0] > 0.5:
    print("Dog")
else:
    print("Cat")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Dog
